DepMap PRISM Dataset

In [1]:
import pandas as pd

In [2]:
cell_info = pd.read_csv("C:/Users/jondor/Downloads/Primary-screen/primary-screen-cell-line-info.csv")

In [3]:
cell_info['primary_tissue'].unique()

array(['esophagus', 'uterus', 'pancreas', 'colorectal', 'liver',
       'central_nervous_system', 'urinary_tract', 'lung', 'rhabdoid',
       'ovary', 'bone', 'mesothelioma', 'skin', 'kidney', 'soft_tissue',
       'thyroid', 'upper_aerodigestive', 'peripheral_nervous_system',
       'gastric', nan, 'bile_duct', 'breast', 'prostate', 'fibroblast',
       'rhabdomyosarcoma'], dtype=object)

In [4]:
# drop rows with NaN in 'primary_tissue'
cell_info = cell_info.dropna(subset=['primary_tissue'])
# selecting gastric and colorectal cell lines
cell_info = cell_info[cell_info['primary_tissue'].str.contains("gastric|colorectal", case=False)]


In [5]:
# load treatment info to map compound names
treatment_info = pd.read_csv("C:/Users/jondor/Downloads/Primary-screen/primary-screen-replicate-treatment-info.csv")

In [6]:
## load LFC data
lfc_data = pd.read_csv("C:/Users/jondor/Downloads/Primary-screen/primary-screen-logfold-change.csv", index_col=0)

In [7]:
print(lfc_data.index)

Index(['ACH-000001', 'ACH-000007', 'ACH-000008', 'ACH-000010_FAILED_STR',
       'ACH-000011', 'ACH-000012', 'ACH-000013', 'ACH-000014', 'ACH-000015',
       'ACH-000018',
       ...
       'ACH-001200', 'ACH-001205', 'ACH-001210', 'ACH-001211', 'ACH-001212',
       'ACH-001239', 'ACH-001306', 'ACH-001307', 'ACH-001318', 'ACH-001321'],
      dtype='object', length=578)


In [8]:
matching_rows = cell_info['depmap_id'][cell_info['depmap_id'].isin(lfc_data.index)]

lfc_data = lfc_data.loc[matching_rows]

In [9]:
matching_cols = treatment_info['column_name'][treatment_info['column_name'].isin(lfc_data.columns)]

In [10]:
treatment_info = treatment_info.loc[treatment_info['column_name'].isin(matching_cols)]

In [11]:
## read file
drug_enrichment = pd.read_excel('C:/Users/jondor/Documents/DT/FAP/FAP experiments and trials/Supplementary Files/Combined_Drug_Enrichment_results.xlsx')

In [12]:
drug_enrichment.head()

,Term,Odds Ratio,-log10_pval,Genes,Dataset
0,LAPATINIB,93.910588,1.838675,CCND1;MET,FdS_Signature
1,PALBOCICLIB,53.307487,1.674741,CCND1;MET,FdS_Signature
2,TAMOXIFEN,39.724826,1.607007,EDN1;CCND1,FdS_Signature
3,LY2835219,277.458333,1.431144,CCND1,FdS_Signature
4,DOXORUBICIN,21.448462,1.431144,EDN1;CCND1,FdS_Signature


In [35]:
# filter treatment_info for rows in drug_enrichment

# Normalize both to lowercase
treatment_names = treatment_info['name'].str.lower()
enrichment_terms = drug_enrichment['Term'].str.lower()
#matched = treatment_info['name'][treatment_info['name'].isin(drug_enrichment['Term'])]

# Match based on lowercase strings
matched = treatment_info[treatment_names.isin(enrichment_terms)]

#treatment_info_filtered = treatment_info.loc[;,matched]

In [14]:
find_lfc = lfc_data.loc[:, matched['column_name']]

In [ ]:
# Mean LFC across all 56 cell lines for each drug
mean_lfc_per_drug = find_lfc.mean(axis=0).sort_values()

# View top 10 most effective drugs (more negative LFC)
print(mean_lfc_per_drug.head(30))

In [ ]:
drug_by_cellline = find_lfc.transpose()

# Create mapping from drug_id to drug_name
id_to_name = treatment_info.set_index('column_name')['name'].to_dict()

# Add drug name column
drug_by_cellline['drug_name'] = drug_by_cellline.index.map(id_to_name)

In [17]:
drug_by_cellline['mean_lfc'] = drug_by_cellline.iloc[:, :-1].mean(axis=1)  # exclude drug_name
drug_by_cellline['median_lfc'] = drug_by_cellline.iloc[:, :-2].median(axis=1)
drug_by_cellline['min_lfc'] = drug_by_cellline.iloc[:, :-3].min(axis=1)
drug_by_cellline['max_lfc'] = drug_by_cellline.iloc[:, :-4].max(axis=1)


In [18]:
summary_stats = drug_by_cellline[['drug_name', 'mean_lfc', 'median_lfc', 'min_lfc', 'max_lfc']].sort_values(by='mean_lfc')
# Save the summary statistics to a CSV file
summary_stats.to_excel('C:/Users/jondor/Documents/DT/FAP/FAP experiments and trials/Supplementary Files/summary_primary_drug_response_prism_stats.xlsx', index=False)

#### Secondary Screen Data

In [13]:
import pandas as pd

In [14]:
second_cell_info = pd.read_csv("C:/Users/jondor/Downloads/Secondary-screen/secondary-screen-cell-line-info.csv")

In [15]:
second_cell_info['primary_tissue'].unique()

array(['esophagus', 'uterus', 'pancreas', 'colorectal', 'liver',
       'central_nervous_system', 'urinary_tract', 'lung', 'rhabdoid',
       'ovary', 'bone', 'mesothelioma', 'skin', 'kidney', 'soft_tissue',
       'thyroid', 'upper_aerodigestive', 'peripheral_nervous_system',
       'gastric', 'bile_duct', 'breast', 'prostate', 'fibroblast',
       'rhabdomyosarcoma', nan], dtype=object)

In [16]:
second_cell_info['secondary_tissue'].unique()

array(['esophagus_squamous', 'uterus_endometrium', nan, 'glioma',
       'lung_NSC', 'colorectal_MSI', 'ovary_non_epithelial',
       'osteosarcoma', 'ovary_adenocarcinoma', 'melanoma',
       'leiomyosarcoma', 'fibrosarcoma', 'upper_aerodigestive_squamous',
       'medulloblastoma', 'lung_small', 'neuroblastoma', 'chondrosarcoma',
       'Ewing_sarcoma', 'gastric_adenosquamous', 'gastric_adenocarcinoma',
       'cholangiocarcinoma', 'hepatoblastoma', 'ERpos', 'TNBC', 'HER2Amp',
       'uterine_sarcoma', 'TPBC', 'renal', 'esophagus_adenocarcinoma',
       'pleomorphic_sarcoma', 'gallbladder', 'thyroid_sarcoma',
       'sarcoma_undifferentiated'], dtype=object)

In [17]:
second_cell_info = second_cell_info.dropna(subset=['primary_tissue'])
# selecting gastric and colorectal cell lines
second_cell_info = second_cell_info[second_cell_info['primary_tissue'].str.contains("gastric|colorectal", case=False)]

In [18]:
# load treatment info to map compound names
second_treatment_info = pd.read_csv("C:/Users/jondor/Downloads/Secondary-screen/secondary-screen-replicate-treatment-info.csv")

second_treatment_info = second_treatment_info[second_treatment_info['perturbation_type'] == 'experimental_treatment']
## load LFC data
second_lfc_data = pd.read_csv("C:/Users/jondor/Downloads/Secondary-screen/secondary-screen-logfold-change.csv", index_col=0)

In [19]:
second_matching_rows = second_cell_info['depmap_id'][second_cell_info['depmap_id'].isin(second_lfc_data.index)]

second_lfc_data = second_lfc_data.loc[second_matching_rows]
second_matching_cols = second_treatment_info['column_name'][second_treatment_info['column_name'].isin(second_lfc_data.columns)]
second_treatment_info = second_treatment_info.loc[second_treatment_info['column_name'].isin(second_matching_cols)]
## read file
drug_enrichment = pd.read_excel('C:/Users/jondor/Documents/DT/FAP/FAP experiments and trials/Supplementary Files/FAP_Polyp_Drug_Enrichment_results.xlsx')


In [20]:
# Normalize both to lowercase
second_treatment_names = second_treatment_info['name'].str.lower()
second_enrichment_terms = drug_enrichment['Term'].str.lower()
#matched = treatment_info['name'][treatment_info['name'].isin(drug_enrichment['Term'])]

# Match based on lowercase strings
second_matched = second_treatment_info[second_treatment_names.isin(second_enrichment_terms)]


In [21]:
second_lfc = second_lfc_data.loc[:, second_matched['column_name']]

In [ ]:
second_drug_by_celline = second_lfc.transpose()

# Create mapping from drug_id to drug_name
id_to_name2 = second_treatment_info.set_index('column_name')['name'].to_dict()

# Add drug name column
second_drug_by_celline['drug_name'] = second_drug_by_celline.index.map(id_to_name2)
second_drug_by_celline['mean_lfc'] = second_drug_by_celline.iloc[:, :-1].mean(axis=1)  # exclude drug_name
second_drug_by_celline['median_lfc'] = second_drug_by_celline.iloc[:, :-2].median(axis=1)
second_drug_by_celline['min_lfc'] = second_drug_by_celline.iloc[:, :-3].min(axis=1)
second_drug_by_celline['max_lfc'] = second_drug_by_celline.iloc[:, :-4].max(axis=1)

summary_stats = second_drug_by_celline[['drug_name', 'mean_lfc', 'median_lfc', 'min_lfc', 'max_lfc']].sort_values(by='mean_lfc')
# Save the summary statistics to a CSV file
summary_stats.to_excel('C:/Users/jondor/Documents/DT/FAP/FAP experiments and trials/Supplementary Files/summary_secondary_drug_response_prism_stats_FAP_Polyp_only.xlsx', index=False)

In [23]:
## filter second_drug_by_celline by mean_lfc < -0.5
second_drug_by_celline  = second_drug_by_celline[second_drug_by_celline['mean_lfc'] < -0.5]

In [24]:
summary_stats['drug_name'].unique()[:10]

array(['doxorubicin', 'bortezomib', 'daunorubicin', 'digoxin', 'ouabain',
       'curcumin', 'gossypol', 'spironolactone', 'epirubicin',
       'digitoxin'], dtype=object)

In [25]:
## create new df from second_drug_by_celline with drug name and number of cell lines for each drug
drug_counts = second_drug_by_celline['drug_name'].value_counts().reset_index()
drug_counts.columns = ['drug_name', 'cell_line_count']
# Save the drug counts to a CSV file
drug_counts.head(10)

,drug_name,cell_line_count
0,talazoparib,29
1,paclitaxel,28
2,ganetespib,27
3,trametinib,26
4,camptothecin,24
5,tanespimycin,24
6,gemcitabine,24
7,sirolimus,23
8,bortezomib,22
9,temsirolimus,22


In [26]:
# Assuming 'drug_name' is a column and LFC columns are numeric
lfc_cols = second_drug_by_celline.columns[:-5]

# Aggregate using apply to compute across all LFC columns for each drug
def aggregate_lfc(group):
    lfc_data = group[lfc_cols]
    return pd.Series({
        'total_non_missing_lfc': lfc_data.notna().sum().sum(),
        'mean_lfc': lfc_data.mean().mean(),
        'median_lfc': lfc_data.median().median(),
        'min_lfc': lfc_data.min().min(),
        'max_lfc': lfc_data.max().max(),
        'cell_line_count': len(group)
    })

drug_df = second_drug_by_celline.groupby('drug_name').apply(aggregate_lfc,include_groups=False).reset_index()


In [27]:
## read file with OR values
or_df = pd.read_excel('C:/Users/jondor/Documents/DT/FAP/FAP experiments and trials/Supplementary Files/FAP_Polyp_Drug_Enrichment_results.xlsx')

In [28]:
or_df = (
    or_df.groupby('Term', as_index=False)
    .agg({
        'Genes': lambda x: ';'.join(x.dropna().unique()),
        'Odds Ratio': 'mean'  # or 'max', 'min', etc.
    })
    .rename(columns={'Genes': 'Genes_Enrichr'})
)


In [29]:
drug_df['drug_name'] = drug_df['drug_name'].str.lower()
or_df['Term'] = or_df['Term'].str.lower()
second_treatment_selected = or_df.drop_duplicates()
drug_df = drug_df.merge(second_treatment_selected, left_on='drug_name', right_on='Term', how='left')

In [ ]:
drug_df.to_excel('C:/Users/jondor/Documents/DT/FAP/FAP experiments and trials/Supplementary Files/secondary_drug_response_summary_FAP_Polyp.xlsx', index=False)

In [30]:
FAP_ligands = ['LAMB1', 'LAMC2', 'TGFBI', 'SEMA4D' ,'AGRN', 'SEMA5A','SEMA3F', 'INHBA', 'LAMA5', 'COL7A1']
## FAP Polyps multinichenet data 
FAP_polyps = pd.read_csv('C:/Users/jondor/Documents/DT/FAP/lr_target_df_2.csv')
FAP_polyps = FAP_polyps[FAP_polyps['group']== 'FAP_Polyp']
FAP_polyps = FAP_polyps[FAP_polyps['ligand'].isin(FAP_ligands)]

In [31]:
# only FAP Polyp
targets_df = pd.DataFrame({
    'FAP_Polyp': pd.Series(FAP_polyps['target'].unique().tolist())
})

In [83]:
# save to excel and parquet
targets_df.to_excel('C:/Users/jondor/Documents/DT/FAP/FAP experiments and trials/Supplementary Files/FAP_sCRC_targets_of_FdS_ligands.xlsx', index=False)
targets_df.to_parquet('C:/Users/jondor/Documents/DT/FAP/FAP experiments and trials/Supplementary Files/FAP_sCRC_targets_of_FdS_ligands.parquet', index=False)

In [ ]:
## read the file with drug targets
#targets_df = pd.read_csv('C:/Users/jondor/Documents/DT/FAP/targets_Epithelial_df.csv')

In [32]:
for index, row in drug_df.iterrows():
    # Get and clean the drug's targets
    drug_targets = str(row['Genes_Enrichr']).split(';')
    drug_targets = [target.strip() for target in drug_targets if target.strip()]
    drug_targets_set = set(drug_targets)

    for col in targets_df.columns:
        condition_targets = targets_df[col].dropna().unique()
        condition_targets_set = set([t.strip() for t in condition_targets])

        # Find intersection
        intersection = drug_targets_set.intersection(condition_targets_set)
        if intersection:
            drug_df.at[index, f"{col}_matches"] = ", ".join(sorted(intersection))
            # Write the count of overlapping targets
            drug_df.at[index, f"{col}_num_matches"] = len(intersection)
        else:
            drug_df.at[index, f"{col}_matches"] = None
            drug_df.at[index, f"{col}_num_matches"] = 0


In [33]:
drug_df.columns

Index(['drug_name', 'total_non_missing_lfc', 'mean_lfc', 'median_lfc',
       'min_lfc', 'max_lfc', 'cell_line_count', 'Term', 'Genes_Enrichr',
       'Odds Ratio', 'FAP_Polyp_matches', 'FAP_Polyp_num_matches'],
      dtype='object')

In [86]:
## save the drug_df with targets 

drug_df[['drug_name','mean_lfc','Genes_Enrichr','FAP_Unaffected_matches','FAP_Polyp_matches','sCRC_matches']].to_excel('C:/Users/jondor/Documents/DT/FAP/FAP experiments and trials/Supplementary Files/enriched_drugs_on_cellines_rank_and_overlap.xlsx', index=False)

In [34]:
pathways_df = {}
for name in ['FAP_Polyp']:
    # read the pathways file
    pathways_df[name] = pd.read_excel(f'C:/Users/jondor/Documents/DT/FAP/FAP experiments and trials/Supplementary Files/{name}_Pathway_results.xlsx')
    ## filter pathways_df[name] for pathways with p-value < 0.05 and  order by Odds Ratio
    pathways_df[name] = pathways_df[name][pathways_df[name]['Adjusted P-value'] < 0.05].sort_values(by='Odds Ratio', ascending=False)
        

In [36]:
records = []

for condition, pathway_data in pathways_df.items():
    # sort pathways by Odds Ratio and limit to top 20
    # This assumes 'Odds Ratio' is a column in pathway_data
    pathway_data = pathway_data.sort_values(by='Odds Ratio', ascending=False)
    limited_pathways = pathway_data.head(20)

    # Get the reference targets set from targets_df
    condition_targets_set = set(targets_df[condition].dropna().astype(str).str.strip())

    for _, drug_row in drug_df.iterrows():
        drug_name = drug_row['drug_name']
        genes_field = drug_row.get('Genes_Enrichr')

        if pd.isna(genes_field):
            continue

        drug_targets = str(genes_field).split(';')
        drug_targets_set = {t.strip() for t in drug_targets if t.strip()}

        for _, pathway_row in limited_pathways.iterrows():
            pathway_name = pathway_row['Term']
            OR = pathway_row['Odds Ratio']
            pathway_targets = str(pathway_row['Genes']).split(';')
            pathway_targets_set = {t.strip() for t in pathway_targets if t.strip()}

            intersection = drug_targets_set.intersection(pathway_targets_set).intersection(condition_targets_set)

            #intersection = drug_targets_set.intersection(pathway_targets_set)
            if not intersection:
                continue

            records.append({
                'drug_name': drug_name,
                'condition': condition,
                'pathway_name': pathway_name,
                'OR': OR,
                'overlapping_genes': ";".join(sorted(intersection)),
                'overlap_count': len(intersection),
                'Enrichr_drug_targets': ";".join(sorted(drug_targets_set))
            })

drug_pathway_df = pd.DataFrame(records)

In [ ]:
# save to csv
drug_pathway_df.to_excel('C:/Users/jondor/Documents/DT/FAP/FAP experiments and trials/Supplementary Files/FAP_Polyp_drug_pathway_overlap_gene_df.xlsx', index= False)